In [4]:
import sqlite3

# Connect to a database (or create it if it doesn't exist)
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Create a table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL
    )
''')

# Insert data

cursor.execute("INSERT INTO users (name, email) VALUES (?, ?)", ("PHat", "Facheng@example.com"))
cursor.execute("INSERT INTO users (name, email) VALUES (?, ?)", ("TS", "TS@email.com"))
# Commit changes
conn.commit()

# Query data
cursor.execute("SELECT * FROM users")
users = cursor.fetchall()

# Print results
for user in users:
    print(f"ID: {user[0]}, Name: {user[1]}, Email: {user[2]}")

# Close connection
conn.close()

ID: 1, Name: Alice, Email: alice@example.com
ID: 2, Name: Bob, Email: bob@example.com
ID: 3, Name: PHat, Email: Facheng@example.com
ID: 4, Name: TS, Email: TS@email.com


In [2]:
from flask import Flask, request, jsonify
import sqlite3

app = Flask(__name__)

# Hàm kết nối đến cơ sở dữ liệu
def get_db_connection():
    conn = sqlite3.connect('example.db')
    conn.row_factory = sqlite3.Row
    return conn

# Tạo bảng users nếu chưa tồn tại
def init_db():
    conn = get_db_connection()
    conn.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL
        )
    ''')
    conn.close()

init_db()

# API endpoints
@app.route('/users', methods=['GET'])
def get_users():
    conn = get_db_connection()
    users = conn.execute('SELECT * FROM users').fetchall()
    conn.close()
    return jsonify([dict(user) for user in users])

@app.route('/users', methods=['POST'])
def create_user():
    user_data = request.json
    conn = get_db_connection()
    try:
        conn.execute('INSERT INTO users (name, email) VALUES (?, ?)',
                     (user_data['name'], user_data['email']))
        conn.commit()
        return jsonify({"message": "User created successfully"}), 201
    except sqlite3.IntegrityError:
        return jsonify({"error": "Email already exists"}), 400
    finally:
        conn.close()

@app.route('/users/<int:user_id>', methods=['GET'])
def get_user(user_id):
    conn = get_db_connection()
    user = conn.execute('SELECT * FROM users WHERE id = ?', (user_id,)).fetchone()
    conn.close()
    if user is None:
        return jsonify({"error": "User not found"}), 404
    return jsonify(dict(user))

@app.route('/users/<int:user_id>', methods=['PUT'])
def update_user(user_id):
    user_data = request.json
    conn = get_db_connection()
    try:
        conn.execute('UPDATE users SET name = ?, email = ? WHERE id = ?',
                     (user_data['name'], user_data['email'], user_id))
        conn.commit()
        if conn.total_changes == 0:
            return jsonify({"error": "User not found"}), 404
        return jsonify({"message": "User updated successfully"})
    except sqlite3.IntegrityError:
        return jsonify({"error": "Email already exists"}), 400
    finally:
        conn.close()

@app.route('/users/<int:user_id>', methods=['DELETE'])
def delete_user(user_id):
    conn = get_db_connection()
    conn.execute('DELETE FROM users WHERE id = ?', (user_id,))
    conn.commit()
    if conn.total_changes == 0:
        conn.close()
        return jsonify({"error": "User not found"}), 404
    conn.close()
    return jsonify({"message": "User deleted successfully"})

if __name__ == '__main__':
    import threading
    def run_app():
        app.run(debug=True, use_reloader=False)
    
    threading.Thread(target=run_app).start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jul/2024 20:14:30] "GET /users HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2024 20:18:10] "GET /users HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2024 20:22:35] "GET / HTTP/1.1" 404 -


In [ ]:
from flask import Flask, request, jsonify, render_template_string
import sqlite3

def create_app():
    app = Flask(__name__)

    # Hàm kết nối đến cơ sở dữ liệu
    def get_db_connection():
        conn = sqlite3.connect('example.db')
        conn.row_factory = sqlite3.Row
        return conn

    # Tạo bảng users nếu chưa tồn tại
    def init_db():
        conn = get_db_connection()
        conn.execute('''
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                email TEXT UNIQUE NOT NULL
            )
        ''')
        conn.close()

    init_db()

    # API endpoints
    @app.route('/users', methods=['GET'])
    def get_users():
        conn = get_db_connection()
        users = conn.execute('SELECT * FROM users').fetchall()
        conn.close()
        return jsonify([dict(user) for user in users])

    @app.route('/users', methods=['POST'])
    def create_user():
        user_data = request.json
        conn = get_db_connection()
        try:
            conn.execute('INSERT INTO users (name, email) VALUES (?, ?)',
                         (user_data['name'], user_data['email']))
            conn.commit()
            return jsonify({"message": "User created successfully"}), 201
        except sqlite3.IntegrityError:
            return jsonify({"error": "Email already exists"}), 400
        finally:
            conn.close()

    @app.route('/users/<int:user_id>', methods=['GET'])
    def get_user(user_id):
        conn = get_db_connection()
        user = conn.execute('SELECT * FROM users WHERE id = ?', (user_id,)).fetchone()
        conn.close()
        if user is None:
            return jsonify({"error": "User not found"}), 404
        return jsonify(dict(user))

    @app.route('/users/<int:user_id>', methods=['PUT'])
    def update_user(user_id):
        user_data = request.json
        conn = get_db_connection()
        try:
            conn.execute('UPDATE users SET name = ?, email = ? WHERE id = ?',
                         (user_data['name'], user_data['email'], user_id))
            conn.commit()
            if conn.total_changes == 0:
                return jsonify({"error": "User not found"}), 404
            return jsonify({"message": "User updated successfully"})
        except sqlite3.IntegrityError:
            return jsonify({"error": "Email already exists"}), 400
        finally:
            conn.close()

    @app.route('/users/<int:user_id>', methods=['DELETE'])
    def delete_user(user_id):
        conn = get_db_connection()
        conn.execute('DELETE FROM users WHERE id = ?', (user_id,))
        conn.commit()
        if conn.total_changes == 0:
            conn.close()
            return jsonify({"error": "User not found"}), 404
        conn.close()
        return jsonify({"message": "User deleted successfully"})

    @app.route('/')
    def home():
        conn = get_db_connection()
        users = conn.execute('SELECT * FROM users').fetchall()
        conn.close()
        
        html = '''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>User Management</title>
            <style>
                table {
                    border-collapse: collapse;
                    width: 100%;
                }
                th, td {
                    border: 1px solid black;
                    padding: 8px;
                    text-align: left;
                }
                th {
                    background-color: #f2f2f2;
                }
            </style>
        </head>
        <body>
            <h1>User Management</h1>
            <table>
                <tr>
                    <th>ID</th>
                    <th>Name</th>
                    <th>Email</th>
                </tr>
                {% for user in users %}
                <tr>
                    <td>{{ user['id'] }}</td>
                    <td>{{ user['name'] }}</td>
                    <td>{{ user['email'] }}</td>
                </tr>
                {% endfor %}
            </table>
        </body>
        </html>
        '''
        
        return render_template_string(html, users=users)

    return app

# Tạo ứng dụng
app = create_app()

# Chạy ứng dụng
if __name__ == '__main__':
    import threading
    def run_app():
        app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)
    
    threading.Thread(target=run_app).start()


!idle

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.5:5000
Press CTRL+C to quit
